In [1]:
import numpy as np
# import os
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import random
from sklearn.utils import shuffle
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as Fabs
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from scipy.io import loadmat

In [86]:
source = loadmat('dataset/B21.mat')
target = loadmat('dataset/B27.mat')
n= 640 # number of interpolated points
source_bridge = np.zeros((2400,2,640))
target_bridge = np.zeros((2400,2,640))
DL_source = np.zeros(2400)
DM_source = np.zeros(2400)
DL_target = np.zeros(2400)
DM_target = np.zeros(2400)
for i in range(source['data'].shape[1]):
    f = source['data'][0][i]
    acc,dl,dm = interpolation(f)
    source_bridge[i,:,:] = acc
    DL_source[i] = dl
    DM_source[i] = dm
for i in range(target['data'].shape[1]):
    f = target['data'][0][i]
    acc,dl,dm = interpolation(f)
    target_bridge[i,:,:] = acc
    DL_target[i] = dl
    DM_target[i] = dm

In [87]:
# Boostrapping the Source and Target Bridge
data_source_train = np.zeros((1500,2,640))
data_source_test = np.zeros((900,2,640))
data_target_train = np.zeros((1500,2,640))
data_target_test = np.zeros((900,2,640))
dl_source_train = np.zeros(1500)
dl_source_test = np.zeros(900)
dl_target_train = np.zeros(1500)
dl_target_test = np.zeros(900)
dm_source_train = np.zeros(1500)
dm_source_test = np.zeros(900)
dm_target_train = np.zeros(1500)
dm_target_test = np.zeros(900)

In [88]:
for i in range(6):
    data_train,data_test,dl,dm = bootstrap(source_bridge[400*i:400*(i+1),:,:],DL_source[400*i:400*(i+1)],DM_source[400*i:400*(i+1)])
    data_source_train[250*i:250*(i+1),:,:] = data_train
    data_source_test[150*i:150*(i+1),:,:] = data_test
    dl_source_train[250*i:250*(i+1)] = dl
    dm_source_train[250*i:250*(i+1)] = dm
    dl_source_test[150*i:150*(i+1)] = dl
    dm_source_test[150*i:150*(i+1)] = dm
for i in range(6):
    data_train,data_test,dl,dm = bootstrap(target_bridge[400*i:400*(i+1),:,:],DL_target[400*i:400*(i+1)],DM_target[400*i:400*(i+1)])
    data_target_train[250*i:250*(i+1),:,:] = data_train
    data_target_test[150*i:150*(i+1),:,:] = data_test
    dl_target_train[250*i:250*(i+1)] = dl
    dm_target_train[250*i:250*(i+1)] = dm
    dl_target_test[150*i:150*(i+1)] = dl
    dm_target_test[150*i:150*(i+1)] = dm

In [89]:
x_source_train,yl_source_train,ys_source_train=shuffle(data_source_train,dl_source_train,dm_source_train)
x_target_train,yl_target_train,ys_target_train=shuffle(data_target_train,dl_target_train,dm_target_train)
x_source_test,yl_source_test,ys_source_test=shuffle(data_source_test,dl_source_test,dm_source_test)
x_target_test,yl_target_test,ys_target_test=shuffle(data_target_test,dl_target_test,dm_target_test)
dataset_train_source = Dataset(x_source_train,yl_source_train,ys_source_train, resize_size=64, \
                              crop_size=62, is_train = True)
dataset_test_source = Dataset(x_source_test,yl_source_test,ys_source_test, resize_size=64, \
                              crop_size=62, is_train = False)
dataset_train_target = Dataset(x_target_train,yl_target_train,ys_target_train, resize_size=64, \
                              crop_size=62, is_train = True)
dataset_test_target = Dataset(x_target_test,yl_target_test,ys_target_test, resize_size=64, \
                              crop_size=62, is_train = False)
    
batch_size = 100
source_train = torch.utils.data.DataLoader(dataset_train_source,batch_size, shuffle=True)
source_train_ = torch.utils.data.DataLoader(dataset_train_source,batch_size, shuffle=True)
source_test = torch.utils.data.DataLoader(dataset_test_source,batch_size, shuffle=False)
target_train = torch.utils.data.DataLoader(dataset_train_target,batch_size, shuffle=True)
target_train_ = torch.utils.data.DataLoader(dataset_train_target,batch_size, shuffle=True)
target_test = torch.utils.data.DataLoader(dataset_test_target,batch_size, shuffle=False)
    
device='cuda'
model = SupDann().to(device)
optimizer = optim.SGD(model.parameters(), lr= 0.025, momentum= 0.9)
criterion = nn.CrossEntropyLoss().to(device)
weight = 1.
gamma = 1.
allepoch=301
CUDA_LAUNCH_BLOCKING=1
losses = []
accuracies = []
f1s = []
losses_save,accuracy_save,f1_save = test_st(0,200,model,source_train,source_test,\
                    target_train,target_test,criterion,\
                    criterion,criterion)
losses = np.append(losses,losses_save)
accuracies=np.append(accuracies,accuracy_save)
f1s=np.append(f1s,f1_save)
for epoch in range(allepoch):
    len_dataloader = min(len(source_train), len(target_train))
    total_steps = allepoch * len(source_train)
    i = 0
    model.train()
    for batch_idx, (data_source, data_source_, \
                    data_target, data_target_) in enumerate(zip(source_train, \
                    source_train_, target_train, target_train_)):
        
        s_img, s_label_l, s_label_s = data_source
        s_img_, s_label_l_, s_label_s_ = data_source_
        start_steps = epoch * len(source_train)
        p = float(i + start_steps) / total_steps
        alpha = 2. / (1. + np.exp(-10 * p)) - 1
        optimizer = optimizer_scheduler(optimizer, p)
        optimizer.zero_grad()

        batch_size = len(s_label_l)

        domain_labels = torch.cat((torch.zeros(batch_size),torch.ones(batch_size)))
        domain_labels = domain_labels.long().to(device)

        # source domain
        input_s = (s_img.to(device),s_img_.to(device))
        a,b,c,d = model(input_s,alpha)
        d_xl_s,d_xq_s,d_xq_hier_s = b
        latent_xl_s,latent_xq_s,latent_xq_hier_s = d
        class_loss_l = criterion(a, s_label_l.to(device))
        class_loss_q = criterion(c, s_label_s_.to(device))
        t_img, t_label_l, t_label_s = data_target
        t_img_, t_label_l_, t_label_s = data_target_

        input_t = (t_img.to(device),t_img_.to(device))
        _, bt, _,dt = model(input_t,alpha)
        d_xl_t,d_xq_t,d_xq_hier_t = bt
        latent_xl_t,latent_xq_t,latent_xq_hier_t = dt

        class_loss = class_loss_l+class_loss_q
        transfer_loss1 = criterion(torch.cat((d_xl_s,d_xl_t),0), domain_labels)
        transfer_loss2 = criterion(torch.cat((d_xq_s,d_xq_t),0), domain_labels)
        transfer_loss3 = criterion(torch.cat((d_xq_hier_s,d_xq_hier_t),0), domain_labels)
        transfer_losses = torch.vstack((transfer_loss1,transfer_loss2))
        transfer_loss = 0.1*torch.mul(torch.div(torch.log(torch.sum(torch.exp(torch.mul(transfer_losses,\
                      gamma)))),gamma),weight)+0.5*transfer_loss3
         
        err = class_loss+transfer_loss
        err.backward()
        optimizer.step()
        i=i+1
    losses_save,accuracy_save,f1_save = test_st(epoch,50,model,source_train,source_test,\
                    target_train,target_test,criterion,\
                    criterion,criterion)
    losses = np.vstack((losses,losses_save))
    accuracies=np.vstack((accuracies,accuracy_save))
    f1s=np.vstack((f1s,f1_save))

Epoch 0
Source test set: Average loss: 1.0986, 1.0987
Target test set: Average loss: 1.0986, 1.0987
Epoch 0
Source test set: Average loss: 1.0981, 1.1000
Target test set: Average loss: 1.0976, 1.1012
Epoch 50
Source test set: Average loss: 0.8203, 1.0370
Target test set: Average loss: 0.7801, 0.9233
Epoch 100
Source test set: Average loss: 0.8039, 0.9421
Target test set: Average loss: 0.6940, 0.8670
Epoch 150
Source test set: Average loss: 0.7686, 0.9184
Target test set: Average loss: 0.6529, 0.8368
Epoch 200
Source test set: Average loss: 0.7471, 0.8207
Target test set: Average loss: 0.6803, 0.8378
Epoch 250
Source test set: Average loss: 0.7651, 0.8518
Target test set: Average loss: 0.6667, 0.8462
Epoch 300
Source test set: Average loss: 0.7505, 0.7934
Target test set: Average loss: 0.6439, 0.8239
